# Chapter 3: Linear Neural Networks - Regression

This notebook covers **Chapter 3** of the Deep Learning in Hebrew book, focusing on regression problems. We'll learn how to build models that can predict continuous values from a given set of examples.

## Overview

This chapter deals with regression problems - how to build a model from a given set of examples that can predict continuous values. The model will learn the relationship between input features and output values.

We'll focus on two main types:
1. **Linear Regression** - for predicting continuous values
2. **Logistic Regression** - for binary classification problems

We'll solve regression problems using simple neural networks, which will serve as a foundation for deeper networks that can handle non-linearly separable data.

---

## Table of Contents

1. [Linear Regression](#1-linear-regression)
   - 1.1 [The Basic Concept](#11-the-basic-concept)
   - 1.2 [Gradient Descent](#12-gradient-descent)
   - 1.3 [Regularization and Cross Validation](#13-regularization-and-cross-validation)
2. [Logistic Regression](#2-logistic-regression)

## Setup and Imports

Let's start by importing the necessary libraries for this chapter.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# 1. Linear Regression

## 1.1 The Basic Concept

The simplest model is **linear regression**. This model tries to find a linear relationship between one or more input features and a dependent variable.

### Mathematical Formulation

Given independent variables $\mathbf{x} \in \mathbb{R}^d$ and a dependent variable $y \in \mathbb{R}$, we model their relationship as:

$$\hat{y} = \mathbf{w}^T \mathbf{x} + b = w_1 x_1 + w_2 x_2 + \cdots + w_d x_d + b$$

where:
- $\mathbf{w} \in \mathbb{R}^d$ are the weights (parameters)
- $b \in \mathbb{R}$ is the bias

### Example: House Price Prediction

For example, to predict house prices, we might use features like:
- House size
- Location
- Number of rooms

The model will learn the weights and bias from known examples, then use them to predict prices for houses with unknown prices but known features.

### Loss Function: Mean Squared Error (MSE)

To build a model that accurately estimates $y$ given input features, we need to find the optimal weights and bias. Since they're unknown, we calculate them using a set of known examples.

First, we define a **loss function** $L(\mathbf{w}, b)$ that determines how good a model's performance is. The loss function is a function of the learned parameters. We want to find the optimal values that minimize this function.

A common loss function is the **Mean Squared Error (MSE)**, which calculates the squared difference between predicted and actual values:

$$L^{(i)}(\mathbf{w}, b) = \frac{1}{2}(y^{(i)} - \hat{y}^{(i)})^2$$

For $n$ known examples, we sum all these differences:

$$L(\mathbf{w}, b) = \frac{1}{n}\sum_{i=1}^{n} \frac{1}{2}(y^{(i)} - \hat{y}^{(i)})^2 = \frac{1}{n}\sum_{i=1}^{n} \frac{1}{2}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)} - b)^2$$

To find the optimal parameters, we need to minimize the loss function:

$$\hat{\mathbf{w}}, \hat{b} \equiv \hat{\boldsymbol{\theta}} = \arg\min L(\mathbf{w}, b)$$

### Closed-Form Solution for Scalar Case

For the scalar case where $d = 1$ (single feature), the linear relationship is $\hat{y} = wx + b$.

The loss function becomes:

$$L(\theta) = \frac{1}{n}\sum_{i=1}^{n} \frac{1}{2}(y^{(i)} - wx^{(i)} - b)^2$$

To find the optimum, we take derivatives and set them to zero:

$$\frac{\partial L}{\partial w} = \frac{1}{n}\sum_{i=1}^{n}(y^{(i)} - wx^{(i)} - b) \cdot (-x^{(i)}) = 0$$

$$\frac{\partial L}{\partial b} = \frac{1}{n}\sum_{i=1}^{n}(y^{(i)} - wx^{(i)} - b) \cdot (-1) = 0$$

This gives us a system of linear equations:

$$w\sum x_i^2 + b\sum x_i = \sum y_i x_i$$

$$w\sum x_i + bn = \sum y_i$$

In matrix form:

$$\begin{bmatrix} \sum x_i^2 & \sum x_i \\ \sum x_i & n \end{bmatrix} \begin{bmatrix} w \\ b \end{bmatrix} = \begin{bmatrix} \sum y_i x_i \\ \sum y_i \end{bmatrix}$$

Let's implement this:

In [ ]:
# Generate synthetic data for demonstration
def generate_linear_data(n=100, w_true=2.0, b_true=1.0, noise_std=0.5):
    """Generate synthetic linear data with noise."""
    np.random.seed(42)
    X = np.random.randn(n, 1) * 2
    y = w_true * X.flatten() + b_true + np.random.randn(n) * noise_std
    return X, y

# Generate data
X, y = generate_linear_data(n=100, w_true=2.0, b_true=1.0, noise_std=0.5)

# Visualize the data
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, label='Data points')
plt.xlabel('X (feature)')
plt.ylabel('y (target)')
plt.title('Synthetic Linear Regression Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Data shape: X={X.shape}, y={y.shape}")
print(f"Sample values: X[0]={X[0,0]:.3f}, y[0]={y[0]:.3f}")

In [ ]:
# Closed-form solution for linear regression (scalar case)
def linear_regression_closed_form(X, y):
    """
    Solve linear regression using closed-form solution.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, 1)
        Input features
    y : array-like, shape (n_samples,)
        Target values
    
    Returns:
    --------
    w : float
        Optimal weight
    b : float
        Optimal bias
    """
    X = X.flatten()
    n = len(X)
    
    # Build the system of equations
    A = np.array([
        [np.sum(X**2), np.sum(X)],
        [np.sum(X), n]
    ])
    
    b_vec = np.array([
        np.sum(y * X),
        np.sum(y)
    ])
    
    # Solve the system
    solution = np.linalg.solve(A, b_vec)
    w, b = solution[0], solution[1]
    
    return w, b

# Fit the model
w_optimal, b_optimal = linear_regression_closed_form(X, y)

print(f"Optimal parameters:")
print(f"  w (weight) = {w_optimal:.4f}")
print(f"  b (bias) = {b_optimal:.4f}")
print(f"\nTrue parameters were: w=2.0, b=1.0")

In [ ]:
# Visualize the fitted line
X_plot = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
y_pred = w_optimal * X_plot.flatten() + b_optimal

plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, label='Data points')
plt.plot(X_plot, y_pred, 'r-', linewidth=2, label=f'Fitted line: y = {w_optimal:.2f}x + {b_optimal:.2f}')
plt.xlabel('X (feature)')
plt.ylabel('y (target)')
plt.title('Linear Regression: Closed-Form Solution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calculate MSE
y_pred_all = w_optimal * X.flatten() + b_optimal
mse = np.mean((y - y_pred_all)**2)
print(f"Mean Squared Error: {mse:.4f}")

### Vectorized Solution

For convenience, we can incorporate the bias into the weight vector:

$$\hat{y} = \mathbf{w}^T \mathbf{x} + b = (\mathbf{w}^T, b) \begin{pmatrix} \mathbf{x} \\ 1 \end{pmatrix} = \tilde{\mathbf{w}}^T \tilde{\mathbf{x}}$$

where $\tilde{\mathbf{w}}, \tilde{\mathbf{x}} \in \mathbb{R}^{d+1}$.

For the vector case with $n$ examples, we have:
- $X_{n \times (d+1)} = (\mathbf{x}^{(1)}, \ldots, \mathbf{x}^{(n)})^T$
- $Y = (y^{(1)}, \ldots, y^{(n)})^T$

The loss function becomes:

$$L(\theta) = \frac{1}{n}\sum_{i=1}^{n} \frac{1}{2}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)})^2$$

Minimizing this is equivalent to minimizing $\|Y - X\mathbf{w}\|^2$:

$$\frac{\partial L}{\partial \mathbf{w}} = \frac{1}{n}\sum_{i=1}^{n}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)}) \cdot (-\mathbf{x}^{(i)}) = 0$$

$$\rightarrow X^T(X\mathbf{w} - Y) = 0$$

$$\hat{\mathbf{w}} = (X^T X)^{-1} X^T Y$$

This is known as the **normal equation**.

In [ ]:
# Vectorized closed-form solution
def linear_regression_vectorized(X, y):
    """
    Solve linear regression using vectorized normal equation.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        Input features
    y : array-like, shape (n_samples,)
        Target values
    
    Returns:
    --------
    w : array, shape (n_features + 1,)
        Optimal weights (including bias as last element)
    """
    # Add bias column (column of ones)
    n_samples = X.shape[0]
    X_augmented = np.hstack([X, np.ones((n_samples, 1))])
    
    # Normal equation: w = (X^T X)^(-1) X^T y
    w = np.linalg.solve(X_augmented.T @ X_augmented, X_augmented.T @ y)
    
    return w

# Test with multiple features
np.random.seed(42)
X_multi = np.random.randn(100, 2) * 2
w_true_multi = np.array([2.0, -1.5])
b_true_multi = 1.0
y_multi = X_multi @ w_true_multi + b_true_multi + np.random.randn(100) * 0.5

# Fit the model
w_optimal_multi = linear_regression_vectorized(X_multi, y_multi)

print("Multi-feature linear regression:")
print(f"Optimal weights: {w_optimal_multi[:-1]}")
print(f"Optimal bias: {w_optimal_multi[-1]}")
print(f"\nTrue weights: {w_true_multi}")
print(f"True bias: {b_true_multi}")

## 1.2 Gradient Descent

For complex problems where the closed-form solution is not feasible, we use **Gradient Descent (GD)**. This is an iterative optimization method that finds the minimum of the loss function.

### How Gradient Descent Works

1. Start with an initial guess for the parameters
2. At each step, move in the direction of the negative gradient
3. The gradient is the derivative of the function, indicating the direction of steepest ascent
4. Moving in the negative gradient direction gives the steepest descent
5. To avoid getting stuck at saddle points, we add a **learning rate** ($\epsilon$)

Formally, for an initial guess $\boldsymbol{\theta}^{(0)}$, at each step we update:

$$\boldsymbol{\theta}^{(t+1)} = \boldsymbol{\theta}^{(t)} - \epsilon \cdot \frac{\partial}{\partial \boldsymbol{\theta}^{(t)}} L(\boldsymbol{\theta}^{(t)})$$

This process continues iteratively until convergence. Since the problem is convex, convergence to the minimum is guaranteed, though it can be slow if the learning rate is too large or too small.

### Gradient Descent for Linear Regression

For linear regression with MSE loss:

$$L(\mathbf{w}, b) = \frac{1}{n}\sum_{i=1}^{n} \frac{1}{2}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)} - b)^2$$

The gradients are:

$$\frac{\partial L}{\partial w_j} = -\frac{1}{n}\sum_{i=1}^{n}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)} - b) \cdot x_j^{(i)}$$

$$\frac{\partial L}{\partial b} = -\frac{1}{n}\sum_{i=1}^{n}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)} - b)$$

In [ ]:
class LinearRegressionGD:
    """
    Linear Regression using Gradient Descent.
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.w = None
        self.b = None
        self.loss_history = []
        
    def fit(self, X, y):
        """
        Train the linear regression model using gradient descent.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,)
            Target values
        """
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.w = np.random.randn(n_features)
        self.b = 0.0
        
        # Gradient descent
        for i in range(self.n_iterations):
            # Predictions
            y_pred = X @ self.w + self.b
            
            # Compute gradients
            dw = -(1/n_samples) * X.T @ (y - y_pred)
            db = -(1/n_samples) * np.sum(y - y_pred)
            
            # Update parameters
            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db
            
            # Store loss
            loss = np.mean((y - y_pred)**2) / 2
            self.loss_history.append(loss)
            
    def predict(self, X):
        """Make predictions."""
        return X @ self.w + self.b

In [ ]:
# Train using gradient descent
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Try different learning rates
learning_rates = [0.001, 0.01, 0.1, 0.5]
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, lr in enumerate(learning_rates):
    model = LinearRegressionGD(learning_rate=lr, n_iterations=1000)
    model.fit(X_train, y_train)
    
    # Plot loss history
    axes[idx].plot(model.loss_history)
    axes[idx].set_title(f'Learning Rate = {lr}')
    axes[idx].set_xlabel('Iteration')
    axes[idx].set_ylabel('Loss')
    axes[idx].grid(True, alpha=0.3)
    
    # Make predictions
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    axes[idx].text(0.5, 0.95, f'MSE: {mse:.4f}\nw: {model.w[0]:.4f}, b: {model.b:.4f}', 
                   transform=axes[idx].transAxes, ha='center', va='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.suptitle('Gradient Descent: Effect of Learning Rate', y=1.02, fontsize=14)
plt.show()

In [ ]:
# Visualize the optimization path in parameter space (for scalar case)
def plot_optimization_path(X, y, learning_rate=0.01, n_iterations=100):
    """Visualize gradient descent optimization path."""
    # Create a grid of parameter values
    w_range = np.linspace(-1, 5, 100)
    b_range = np.linspace(-2, 4, 100)
    W, B = np.meshgrid(w_range, b_range)
    
    # Compute loss for each point
    Loss = np.zeros_like(W)
    for i in range(len(w_range)):
        for j in range(len(b_range)):
            y_pred = W[j, i] * X.flatten() + B[j, i]
            Loss[j, i] = np.mean((y - y_pred)**2) / 2
    
    # Run gradient descent
    w = 0.0
    b = 0.0
    w_path = [w]
    b_path = [b]
    
    for _ in range(n_iterations):
        y_pred = w * X.flatten() + b
        dw = -np.mean((y - y_pred) * X.flatten())
        db = -np.mean(y - y_pred)
        w -= learning_rate * dw
        b -= learning_rate * db
        w_path.append(w)
        b_path.append(b)
    
    # Plot
    plt.figure(figsize=(12, 8))
    contour = plt.contour(W, B, Loss, levels=20, alpha=0.6)
    plt.clabel(contour, inline=True, fontsize=8)
    plt.plot(w_path, b_path, 'r-o', markersize=4, linewidth=2, label='GD Path')
    plt.plot(w_path[0], b_path[0], 'go', markersize=10, label='Start')
    plt.plot(w_path[-1], b_path[-1], 'ro', markersize=10, label='End')
    plt.plot(w_optimal, b_optimal, 'b*', markersize=15, label='Optimal (closed-form)')
    plt.xlabel('Weight (w)')
    plt.ylabel('Bias (b)')
    plt.title('Gradient Descent Optimization Path')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_optimization_path(X_train, y_train, learning_rate=0.05, n_iterations=50)

## 1.3 Regularization and Cross Validation

One of the main challenges in regression is **overfitting** - when a model performs well on training data but poorly on new, unseen data (test set).

Models can suffer from two types of bias:
- **Overfitting**: The model fits too closely to training data, often using a high-order model with high variance
- **Underfitting**: The model is too simple and cannot capture the underlying pattern

### Regularization

**Regularization** helps prevent overfitting by adding a penalty term to the loss function. Common regularization techniques include:

1. **L2 Regularization (Ridge Regression)**:
   $$L(\mathbf{w}, b) = \frac{1}{n}\sum_{i=1}^{n} \frac{1}{2}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)} - b)^2 + \frac{\lambda}{2}\|\mathbf{w}\|^2$$

2. **L1 Regularization (Lasso Regression)**:
   $$L(\mathbf{w}, b) = \frac{1}{n}\sum_{i=1}^{n} \frac{1}{2}(y^{(i)} - \mathbf{w}^T \mathbf{x}^{(i)} - b)^2 + \lambda\|\mathbf{w}\|_1$$

where $\lambda$ is the regularization strength.

### Cross Validation

**Cross-validation** is a technique to assess model performance and select hyperparameters. The most common method is **k-fold cross-validation**, where the data is split into k folds, and the model is trained k times, each time using k-1 folds for training and 1 fold for validation.

In [ ]:
class RidgeRegression:
    """
    Ridge Regression (L2 Regularization) using Gradient Descent.
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000, lambda_reg=0.1):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.lambda_reg = lambda_reg
        self.w = None
        self.b = None
        self.loss_history = []
        
    def fit(self, X, y):
        """Train the ridge regression model."""
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.w = np.random.randn(n_features)
        self.b = 0.0
        
        # Gradient descent with L2 regularization
        for i in range(self.n_iterations):
            # Predictions
            y_pred = X @ self.w + self.b
            
            # Compute gradients (with L2 penalty)
            dw = -(1/n_samples) * X.T @ (y - y_pred) + self.lambda_reg * self.w
            db = -(1/n_samples) * np.sum(y - y_pred)
            
            # Update parameters
            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db
            
            # Store loss (with regularization term)
            mse = np.mean((y - y_pred)**2) / 2
            reg_term = (self.lambda_reg / 2) * np.sum(self.w**2)
            loss = mse + reg_term
            self.loss_history.append(loss)
            
    def predict(self, X):
        """Make predictions."""
        return X @ self.w + self.b

In [ ]:
# Demonstrate overfitting and regularization
# Generate data with some noise
np.random.seed(42)
X_poly = np.linspace(-3, 3, 30).reshape(-1, 1)
y_poly = 0.5 * X_poly.flatten()**2 + X_poly.flatten() + 1 + np.random.randn(30) * 0.5

# Split data
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(
    X_poly, y_poly, test_size=0.3, random_state=42
)

# Try different regularization strengths
lambda_values = [0.0, 0.1, 1.0, 10.0]
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, lambda_reg in enumerate(lambda_values):
    model = RidgeRegression(learning_rate=0.01, n_iterations=2000, lambda_reg=lambda_reg)
    model.fit(X_train_poly, y_train_poly)
    
    # Plot
    X_plot = np.linspace(X_poly.min(), X_poly.max(), 100).reshape(-1, 1)
    y_pred_plot = model.predict(X_plot)
    
    axes[idx].scatter(X_train_poly, y_train_poly, alpha=0.6, label='Train', color='blue')
    axes[idx].scatter(X_test_poly, y_test_poly, alpha=0.6, label='Test', color='red', marker='x')
    axes[idx].plot(X_plot, y_pred_plot, 'g-', linewidth=2, label='Model')
    axes[idx].set_title(f'λ = {lambda_reg}')
    axes[idx].set_xlabel('X')
    axes[idx].set_ylabel('y')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    
    # Calculate errors
    train_mse = mean_squared_error(y_train_poly, model.predict(X_train_poly))
    test_mse = mean_squared_error(y_test_poly, model.predict(X_test_poly))
    axes[idx].text(0.05, 0.95, f'Train MSE: {train_mse:.3f}\nTest MSE: {test_mse:.3f}', 
                   transform=axes[idx].transAxes, ha='left', va='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.suptitle('Effect of L2 Regularization (Ridge Regression)', y=1.02, fontsize=14)
plt.show()

In [ ]:
# K-fold Cross Validation
from sklearn.model_selection import KFold

def k_fold_cross_validation(X, y, model_class, k=5, **model_params):
    """
    Perform k-fold cross-validation.
    
    Parameters:
    -----------
    X : array-like
        Features
    y : array-like
        Targets
    model_class : class
        Model class to instantiate
    k : int
        Number of folds
    **model_params : dict
        Parameters to pass to model
    
    Returns:
    --------
    cv_scores : list
        List of MSE scores for each fold
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    cv_scores = []
    
    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        # Train model
        model = model_class(**model_params)
        model.fit(X_train_fold, y_train_fold)
        
        # Evaluate
        y_pred = model.predict(X_val_fold)
        mse = mean_squared_error(y_val_fold, y_pred)
        cv_scores.append(mse)
    
    return cv_scores

# Test different lambda values using cross-validation
lambda_candidates = [0.0, 0.01, 0.1, 1.0, 10.0]
cv_results = []

for lambda_reg in lambda_candidates:
    scores = k_fold_cross_validation(
        X_poly, y_poly, 
        RidgeRegression, 
        k=5,
        learning_rate=0.01,
        n_iterations=2000,
        lambda_reg=lambda_reg
    )
    cv_results.append({
        'lambda': lambda_reg,
        'mean_mse': np.mean(scores),
        'std_mse': np.std(scores),
        'scores': scores
    })

# Plot results
means = [r['mean_mse'] for r in cv_results]
stds = [r['std_mse'] for r in cv_results]
lambdas = [r['lambda'] for r in cv_results]

plt.figure(figsize=(10, 6))
plt.errorbar(lambdas, means, yerr=stds, marker='o', capsize=5, capthick=2)
plt.xlabel('Regularization Strength (λ)')
plt.ylabel('Mean Squared Error')
plt.title('5-Fold Cross-Validation: Selecting Optimal λ')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.show()

# Find best lambda
best_idx = np.argmin(means)
print(f"Best λ: {lambdas[best_idx]}")
print(f"Mean CV MSE: {means[best_idx]:.4f} ± {stds[best_idx]:.4f}")

In [ ]:
# Visualize the sigmoid function
def sigmoid(z):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))  # Clip to avoid overflow

z = np.linspace(-10, 10, 100)
sigma_z = sigmoid(z)

plt.figure(figsize=(10, 6))
plt.plot(z, sigma_z, 'b-', linewidth=2, label='σ(z) = 1/(1 + e^(-z))')
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='Decision boundary (0.5)')
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('z')
plt.ylabel('σ(z)')
plt.title('Sigmoid (Logistic) Function')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
class LogisticRegression:
    """
    Logistic Regression for binary classification using Gradient Descent.
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.w = None
        self.b = None
        self.loss_history = []
        
    def sigmoid(self, z):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
        
    def fit(self, X, y):
        """
        Train the logistic regression model.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,)
            Binary labels (0 or 1)
        """
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.w = np.random.randn(n_features) * 0.01
        self.b = 0.0
        
        # Gradient descent
        for i in range(self.n_iterations):
            # Forward pass
            z = X @ self.w + self.b
            y_pred = self.sigmoid(z)
            
            # Compute gradients
            dw = (1/n_samples) * X.T @ (y_pred - y)
            db = (1/n_samples) * np.sum(y_pred - y)
            
            # Update parameters
            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db
            
            # Store loss (binary cross-entropy)
            loss = -np.mean(y * np.log(y_pred + 1e-15) + (1 - y) * np.log(1 - y_pred + 1e-15))
            self.loss_history.append(loss)
            
    def predict_proba(self, X):
        """Predict probabilities."""
        z = X @ self.w + self.b
        return self.sigmoid(z)
        
    def predict(self, X, threshold=0.5):
        """Predict binary labels."""
        return (self.predict_proba(X) >= threshold).astype(int)

In [ ]:
# Generate synthetic binary classification data
def generate_classification_data(n=200, n_features=2, random_state=42):
    """Generate synthetic binary classification data."""
    np.random.seed(random_state)
    
    # Create two classes
    n_per_class = n // 2
    
    # Class 0: centered at (-1, -1)
    X0 = np.random.randn(n_per_class, n_features) + np.array([-1, -1])
    y0 = np.zeros(n_per_class)
    
    # Class 1: centered at (1, 1)
    X1 = np.random.randn(n_per_class, n_features) + np.array([1, 1])
    y1 = np.ones(n_per_class)
    
    # Combine
    X = np.vstack([X0, X1])
    y = np.hstack([y0, y1])
    
    # Shuffle
    indices = np.random.permutation(n)
    X = X[indices]
    y = y[indices]
    
    return X, y

# Generate and visualize data
X_clf, y_clf = generate_classification_data(n=200, n_features=2)

plt.figure(figsize=(10, 6))
plt.scatter(X_clf[y_clf == 0, 0], X_clf[y_clf == 0, 1], alpha=0.6, label='Class 0', s=50)
plt.scatter(X_clf[y_clf == 1, 0], X_clf[y_clf == 1, 1], alpha=0.6, label='Class 1', s=50)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Binary Classification Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Train logistic regression model
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42
)

model_lr = LogisticRegression(learning_rate=0.1, n_iterations=1000)
model_lr.fit(X_train_clf, y_train_clf)

# Plot loss history
plt.figure(figsize=(10, 6))
plt.plot(model_lr.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Binary Cross-Entropy Loss')
plt.title('Logistic Regression: Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

# Evaluate
y_pred_clf = model_lr.predict(X_test_clf)
accuracy = accuracy_score(y_test_clf, y_pred_clf)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
# Visualize decision boundary
def plot_decision_boundary(X, y, model, title="Decision Boundary"):
    """Plot decision boundary for 2D classification."""
    # Create a mesh
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Predict on mesh
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, levels=50, alpha=0.6, cmap='RdYlBu')
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2, linestyles='--')
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', alpha=0.8, label='Class 0', s=50, edgecolors='black')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='red', alpha=0.8, label='Class 1', s=50, edgecolors='black')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.legend()
    plt.colorbar(label='Probability of Class 1')
    plt.grid(True, alpha=0.3)
    plt.show()

plot_decision_boundary(X_clf, y_clf, model_lr, "Logistic Regression Decision Boundary")

## Summary

In this chapter, we've covered:

1. **Linear Regression**:
   - Basic concept and mathematical formulation
   - Closed-form solution using normal equations
   - Gradient descent optimization
   - Regularization (Ridge regression)
   - Cross-validation for model selection

2. **Logistic Regression**:
   - Binary classification using sigmoid function
   - Binary cross-entropy loss
   - Gradient descent for logistic regression
   - Decision boundary visualization

### Key Takeaways

- **Linear regression** predicts continuous values using a linear relationship between features and target
- **Gradient descent** is an iterative optimization method that finds optimal parameters by following the negative gradient
- **Regularization** helps prevent overfitting by penalizing large weights
- **Cross-validation** is essential for selecting hyperparameters and assessing model performance
- **Logistic regression** extends linear models to binary classification using the sigmoid activation function

These concepts form the foundation for understanding neural networks, which we'll explore in later chapters.